In [1]:
%%writefile score.py
import json
import numpy as np
import os
import time
import requests
import uuid

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
from ibm_watson_openscale.supporting_classes.enums import *


def convert_user_input_2_openscale(input_data):
    users_records = input_data['input']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}


def convert_user_output_2_openscale(output_data):
    output_data = json.loads(output_data)
    users_records = output_data['output']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}


def init():
    global openscale_client
    global cloud_api_key
    global openscale_subscription_uid
    global openscale_subscription
    global scoring_endpoint
    global scoring_headers
    global payload_data_set_id
    
    
    scoring_endpoint = '***' #deployment scoring endpoint
    scoring_headers = {'Content-Type': 'application/json'}
    cloud_api_key = 'aQ1nvgb_q5xOSz_--alW2JyTm7TlSl-cOYnbw_BR75hd' # Cloud API key
    openscale_subscription_uid = "435bae86556949861ec474ac1d87ed42" # subscription id
    authenticator = IAMAuthenticator(apikey=cloud_api_key)
    openscale_client = APIClient(authenticator=authenticator)
    payload_data_set_id = openscale_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=openscale_subscription_uid, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
    
    
def run(input_data):
   
    # ------ CALL SCORING ENDPOINT --------------
    if type(input_data) is str:
        input_data = json.loads(input_data)

    start_time = time.time()        
    response = requests.post(scoring_endpoint, json=input_data, headers=scoring_headers)
    response_time = int((time.time() - start_time)*1000)
    output_data = response.json()

    # ------ PAYLOAD COVERSION TO OPENSCALE FORMAT and LOGGING --------------
    try:
        
        openscale_input = convert_user_input_2_openscale(input_data)
        openscale_output = convert_user_output_2_openscale(output_data)  
        
        records_list=[PayloadRecord(
               request=openscale_input,
               response=openscale_output,
               response_time=response_time
           )]
        openscale_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=records_list)
        
    
    except Exception as e:
        error = str(e)
        print(error)
    finally:
        return output_data
        
        

Writing score.py
